In [1]:
#SAMPLING USER TWEETS FROM SPIRS HISTORY AND CREATING THEIR REPRESENTATION

import zipfile
import pandas as pd
from csv import reader
import re
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import numpy as np
import csv
import random

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def remove_punctuation(text):
    if(type(text)==float):
        return text
    
    ans=""  
    for i in text:     
        if i not in string.punctuation:
            ans+=i    
            
    return ans

In [3]:
EMOJI_DESCRIPTION_SCRUB = re.compile(r':(\S+?):')
HASHTAG_BEFORE = re.compile(r'#(\S+)')
FIND_MENTIONS = re.compile(r'@(\S+)')
LEADING_NAMES = re.compile(r'^\s*((?:@\S+\s*)+)')
TAIL_NAMES = re.compile(r'\s*((?:@\S+\s*)+)$')

In [4]:
from gensim.parsing.preprocessing import remove_stopwords
import emoji
import string

In [5]:
def process_tweet(s, keep_emoji=True, keep_usernames=False):

    s = s.lower()

    #removing urls, htmls tags, etc
    s = re.sub(r'https\S+', r'', str(s))
    s = re.sub(r'\\n', ' ', s)
    s = re.sub(r'\s', ' ', s)
    s = re.sub(r'<br>', ' ', s)
    s = re.sub(r'&amp;', '&', s)
    s = re.sub(r'&#039;', "'", s)
    s = re.sub(r'&gt;', '>', s)
    s = re.sub(r'&lt;', '<', s)
    s = re.sub(r'\'', "'", s)

    #removing stopwords
    s = remove_stopwords(s)

    #removing emojis
    if keep_emoji:
        s = emoji.demojize(s)
    else:
        emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                    "]+", re.UNICODE)

        s = emoj.sub(r'',s)

    #   s = re.sub(r"\\x[0-9a-z]{2,3,4}", "", s)

    #removing hashtags
    s = re.sub(HASHTAG_BEFORE, r'\1!!', s)


    #removing usernames

    #removing just @sign
    if keep_usernames:
        s = ' '.join(s.split())

        s = re.sub(LEADING_NAMES, r' ', s)
        s = re.sub(TAIL_NAMES, r' ', s)

        s = re.sub(FIND_MENTIONS, r'\1', s)

    #removing username completely
    else:
        s = re.sub(FIND_MENTIONS, r' ', s)
    
    #removing username tags - just in case ??
    s = re.sub(re.compile(r'@(\S+)'), r'@', s)
    user_regex = r".?@.+?( |$)|<@mention>"    
    s = re.sub(user_regex," @user ", s, flags=re.I)

    # Just in case -- remove any non-ASCII and unprintable characters, apart from whitespace  
    s = "".join(x for x in s if (x.isspace() or (31 < ord(x) < 127)))
    s = ' '.join(s.split())
    
    s = remove_punctuation(s)

    return s

In [6]:
users1 = pd.read_csv('split/x_train.csv')
users2 = pd.read_csv('split/x_val.csv')
users3 = pd.read_csv('split/x_test.csv')

In [7]:
users = []

for i in range(len(users1['sar_user'])):
    user = users1['sar_user'][i].split('|')
    users.append(int(user[-1]))
    
for i in range(len(users2['sar_user'])):
    user = users2['sar_user'][i].split('|')
    users.append(int(user[-1]))
    
for i in range(len(users3['sar_user'])):
    user = users3['sar_user'][i].split('|')
    users.append(int(user[-1]))

In [8]:
len(users)

23983

In [9]:
#removing duplicate users
users = list(dict.fromkeys(users))

In [10]:
len(users)

22952

In [26]:
#sampling tweets
file = None

with zipfile.ZipFile('/../../../data/sarcasm_data/spirs_history.zip') as zip:
    file = zip.open('spirs_history/SPIRS-non-sarcastic-history.txt', mode='r')

tweets_array = []

i = 0

user_tweets = []
old_user = -1

for line in file:

    try:

        user_id, tweet_id, tweet = re.split(r'\t+', line.decode('utf-8'))

        if int(user_id) in users:

            if i == 0:
                old_user = user_id


            #new user
            if user_id != old_user:

                if len(user_tweets) > 100 :
                    user_tweets = random.sample(user_tweets, 100)

                for t in user_tweets:
                    tweets_array.append(t)

                user_tweets = []
                old_user = user_id

            else:
                user_tweets.append([tweet_id, process_tweet(tweet), user_id])

    except:
        print(re.split(r'\t+', line.decode('utf-8')))

    i += 1

In [1]:
#len(tweets_array)

In [28]:
#creating tweets representation
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

from csv import writer


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch.cuda.set_device(2)
print(torch.cuda.current_device())
print(device)
model = model.to(device)

out_file = '/../../../data/sarcasm_data/user_embeddings_3_final.csv'

with open(out_file, 'a') as f:

    length = len(tweets_array)
    i = 0
    
    while i < length:
        
        if i + 256 < length :
            temp = tweets_array[i : i + 256]
        else :
            temp = tweets_array[i : length]
            
        tweets=list(np.reshape(temp, (len(temp),3))[:,1])
        
        #row[0] - tweet_id, row[1] - tweet, row[2] - user_id/label

        encoded_input = tokenizer(tweets, padding=True, truncation=True, return_tensors='pt').to(device)

        # Compute token embeddings
        with torch.no_grad():
            model_output = model(**encoded_input)

        # Perform pooling
        sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        
        final_tensor = torch.Tensor.cpu(sentence_embeddings)

        writer = csv.writer(f)
    
        for j in range(0,len(temp)):
            tweet_id = temp[j][0]
            embedding = final_tensor[j].numpy()
            user_id = temp[j][2]
            writer.writerow([tweet_id, np.array_str(embedding, max_line_width=np.inf), user_id])

        if i % 102400 == 0:
            print(i)
        i += 256

2
cuda:0
0
